# 0. mimic LOB

Steps before running this notebook : 
1. Launch the server in the '5000' port by running the batch 'LaunchSERVER.cmd' and entering 5000 as port.
2. Visit the server address to make sure the server is running

# 1. Imports

In [1]:
import mimicLOB as lob
from mimicLOB.agent.genericAgent import genericAgent

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pprint import pprint

# local server
localserver = "http://127.0.0.1:5000"

# distant server
distantserver = "http://mimiclob.appspot.com"

# Server used
server = localserver

# 2. Agents
The agents is constantly connected to the distant LOB via the server address.

#### Parameters :
1. distant : if true, the agent must have the server address. If False, he should have the orderbook object passed in the configuration.
2. server : when distant is True, the http address of the server the LOB is running on.
3. id : optional, default is generic. the if is used in the transaction tape.
4. b_record : optional, boolean, default is False. If True, the agent records all his activity : sent orders, executed trades, pending orders ...


In [2]:
EURONEXT_config = {'distant' : True,
                   'server'  : server,
                   'id'      : 'market'}
                 
myself_config = {'distant' : True,
                 'server'  : server,
                 'id'      : 'Fayçal',
                 'b_record': True}

other_config = {'distant'  : True,
                'server'   : server,
                'id'       : 'Olivier'}

euronext = genericAgent(**EURONEXT_config)
myself   = genericAgent(**myself_config)
other    = genericAgent(**other_config)

# 3. Get the LOB state
Either agents have access to LOB information

In [3]:
LOBstate = myself.getLOBState()
tickSize =  myself.getTickSize()
display(f'tick size : {tickSize}')
if type(LOBstate) == pd.DataFrame:
    if len(LOBstate) >0 :
        LOBstate = LOBstate.set_index('Price').sort_index()
        minPrice = LOBstate.index[0]
        maxPrice = LOBstate.index[-1]
        LOBstate = LOBstate.reindex(range(minPrice, maxPrice+tickSize, tickSize))
        LOBstate.plot.bar(figsize=(20, 7))
else:
    display(LOBstate)

'tick size : 1'

# 4. Send orders

#### limit sell order @101 & limit buy order @99
Check the LOB state afterwards

In [4]:
# Send Orders
myself.send_sell_limit_order(quantity = 100,
                             price    = 101)

myself.send_buy_limit_order(quantity = 100,
                             price    = 100)
print('Sent orders : ')
display(pd.DataFrame.from_dict(myself.sentorders).T)

print('\n\nPending orders (Orders that are still in the LOB) : ')
display(pd.DataFrame.from_dict(myself.pendingorders).T)

print('\n\nExecuted Trades : ')
display(pd.DataFrame.from_dict(myself.executedtrades).T)

Sent orders : 


,type,side,quantity,price,trader_id
0,limit,ask,100,101,Fayçal
1,limit,bid,100,100,Fayçal




Pending orders (Orders that are still in the LOB) : 


,order_id,price,quantity,side,timestamp,trader_id,type
1,1,101,100,ask,1586599869327736600,Fayçal,limit
2,2,100,100,bid,1586599869332005800,Fayçal,limit




Executed Trades : 


""


#### match orders
'Other' will sell 50 at 100. A transaction will happen.

In [5]:
other.send_sell_limit_order(quantity = 50,
                             price    = 100)

print('Sent orders : ')
display(pd.DataFrame.from_dict(myself.sentorders).T)

print('\n\nPending orders (Orders that are still in the LOB) : ')
display(pd.DataFrame.from_dict(myself.pendingorders).T)

print('\n\nExecuted Trades : ')
display(pd.DataFrame.from_dict(myself.executedtrades).T)

Sent orders : 


,type,side,quantity,price,trader_id
0,limit,ask,100,101,Fayçal
1,limit,bid,100,100,Fayçal




Pending orders (Orders that are still in the LOB) : 


,order_id,price,quantity,side,timestamp,trader_id,type
1,1,101,100,ask,1586599869327736600,Fayçal,limit
2,2,100,100,bid,1586599869332005800,Fayçal,limit




Executed Trades : 


""


In [7]:
other.executedtrades

SortedDict({1: {'order_id': 1, 'price': 101, 'quantity': 100, 'side': 'ask', 'timestamp': 1586599869327736600, 'trader_id': 'Fayçal', 'type': 'limit'}, 2: {'order_id': 2, 'price': 100, 'quantity': 100, 'side': 'bid', 'timestamp': 1586599869332005800, 'trader_id': 'Fayçal', 'type': 'limit'}})

# 5. Modify orders

Rules for modification : 
1. If the order is already executed : no new order
2. If the quantity is reduced : the order looses priority  
3. If the price is changed : the order looses priority
4. If the quantity is raised : the order keeps priority

Rules for cancellation :
1. If the order is already executed : no order cancellation

#### Cancel an order

In [30]:
print('\n\nPending orders (Orders that are still in the LOB) : ')
display(pd.DataFrame.from_dict(myself.pendingorders).T)



Pending orders (Orders that are still in the LOB) : 


,order_id,price,quantity,side,timestamp,trader_id,type
4,4,101,100,ask,1586539147009621700,Fayçal,limit
5,5,100,100,bid,1586539147014609400,Fayçal,limit
6,6,101,100,ask,1586539182272586300,Fayçal,limit
7,7,100,100,bid,1586539182277572500,Fayçal,limit


In [29]:
myself.cancelOrder('ask', 5)

'CANCELED'

#### modify an order

In [ ]:

myself.modifyOrder('ask', 5)

In [ ]:
print('\n\nPending orders (Orders that are still in the LOB) : ')
display(pd.DataFrame.from_dict(myself.pendingorders).T)

# 6. Modify The LOB
One agent type (market) can reset the LOB, dump the tape, and modify the LOB properties 
#### Change tick size

#### Remove all pending orders

#### Remove orders and transactions

In [14]:
euronext.resetLOB()

'DONE'

#### limit buy orders @100 & 99 & 94

In [65]:
limitbuyorders =  [{'type'      : 'limit', 
                   'side'      : 'bid', 
                   'quantity'  : 50,
                   'price'     : 100,
                   'trader_id'  : 2},
                  {'type'      : 'limit', 
                   'side'      : 'bid', 
                   'quantity'  : 50,
                   'price'     : 99,
                   'trader_id'  : 3},
                  {'type'      : 'limit', 
                   'side'      : 'bid', 
                   'quantity'  : 50,
                   'price'     : 94,
                   'trader_id'  : 4}]
for order_ in limitbuyorders:
    response = requests.get(f"{server}/sendOrder", 
                            json=order_).json()
    trades        = response['trades']
    pendingOrders = response['pendingOrders']
    if pendingOrders is not None:
        pendingtrades[pendingOrders['order_id']] = pendingOrders
    if trades is not None:
        for trade in trades:
            executedtrades[trade['party2'][0]] = trade  
    display(response['status'])

'SENT'

'SENT'

'SENT'

In [66]:
# This one will be executed, and some orders will remain in the book
Marketablelimitbuyorder =  {'type'     : 'limit', 
                           'side'      : 'bid', 
                           'quantity'  : 150,
                           'price'     : 101,
                           'trader_id'  : 521}

response      = requests.get(f"{server}/sendOrder", 
                        json=Marketablelimitbuyorder).json()
trades        = response['trades']
pendingOrders = response['pendingOrders']
if pendingOrders is not None:
    pendingtrades[pendingOrders['order_id']] = pendingOrders
if trades is not None:
    for trade in trades:
        executedtrades[trade['party2'][0]] = trade       
response['status']  

'SENT'

In [67]:
# This one will be executed, and some orders will remain in the book
Marketablelimitsellorder =  {'type'     : 'limit', 
                           'side'      : 'ask', 
                           'quantity'  : 100,
                           'price'     : 101,
                           'trader_id'  : 522}

response      = requests.get(f"{server}/sendOrder", 
                        json=Marketablelimitsellorder).json()
trades        = response['trades']
pendingOrders = response['pendingOrders']
if pendingOrders is not None:
    pendingtrades[pendingOrders['order_id']] = pendingOrders
if trades is not None:
    for trade in trades:
        executedtrades[trade['party2'][0]] = trade     
response['status']   

'SENT'

In [68]:
pprint(executedtrades)

{521: {'party1': [0, 'ask', 1, None],
       'party2': [521, 'bid', None, None],
       'price': 101,
       'quantity': 100,
       'time': 6,
       'timestamp': 6},
 522: {'party1': [521, 'bid', 6, None],
       'party2': [522, 'ask', None, None],
       'price': 101,
       'quantity': 50,
       'time': 7,
       'timestamp': 7}}


 # Cancel orders if they are too far away

In [73]:
# tick size
tickSize = requests.get(f"{server}/getticksize").json()['ticksize']
keys_ = pendingtrades.keys()
for key_ in keys_:
    order = pendingtrades[key_]
    
    # if they are executed    
    #if they too far away
    side = order['side']
    price = order['price']
    
    
    # best price at side
    if side == 'bid':
        bestPrice = requests.get(f"{server}/getbestbid").json()['bestbid']
    else:
        bestPrice = requests.get(f"{server}/getbestask").json()['bestask']
    
    # if best price < 3 * tickSize, cancel
    if (((side=='bid') & (price < bestPrice - 3*tickSize)) | ((side=='ask') & (price > bestPrice + 3*tickSize))):
            print('CANCELING : ')
            pprint(order)
            params = {'side':side, 'id':key_}
            response = requests.get(f"{server}/cancelOrder",
                                    json=params).json()
            display(response['status'])
            del pendingtrades[key_]

CANCELING : 
{'order_id': 5,
 'price': 94,
 'quantity': 50,
 'side': 'bid',
 'timestamp': 5,
 'trader_id': 4,
 'type': 'limit'}


'CANCELED'

In [ ]:
list(pendingtrades.keys())

# get your current book value

In [ ]:
executedtrades

In [ ]:
# Cancel order because it is too far from the best bid
# and remove it from pending orders


In [77]:
executedtrades

SortedDict({521: {'party1': [0, 'ask', 1, None], 'party2': [521, 'bid', None, None], 'price': 101, 'quantity': 100, 'time': 6, 'timestamp': 6}, 522: {'party1': [521, 'bid', 6, None], 'party2': [522, 'ask', None, None], 'price': 101, 'quantity': 50, 'time': 7, 'timestamp': 7}})